In [ ]:
import pandas as pd
import numpy as np

data = pd.read_csv("./FGDD.csv")

# Generate CSV files and import into neo4j

In [2]:
hp_features = data.filter(regex='^HP')
hp_features = hp_features.columns
hpo_nodes = pd.DataFrame({
    "HpoId": hp_features.tolist(),
    "Facialphenotypename": hp_features.tolist(),
    #":LABEL": "FacialPhenotype"
})

disease_feature = data[["Disease_id", "Disease_name"]].dropna(axis=0).drop_duplicates()
disease_id = disease_feature['Disease_id']
disease_name = disease_feature['Disease_name']
disease_nodes = pd.DataFrame({
    "DiseaseId": disease_id,
    "name": disease_name ,
    #":LABEL": "Disease"
})

patient_id = data['patient_id']
pmid = data['pmid']
patient_number = data['patient_number']
gender = data['gender']
age = data['age']
region = data['region']
race = data['race']

patient_nodes = pd.DataFrame({
    "PatientId": patient_id,
    "pmid": pmid,
    "patient_number": patient_number,
    "gender": gender,
    "age": age,
    "region": region,
    "race": race,
    #":LABEL": "Patient"
})


var_feature1 = data[["Variant_Gene_id_1", "Variant_Gene_name_1", "Variant_Gene_details_1", "Variant_Gene_chromosome_name_1", "Variant_Gene_chromosome_location_1", "Variant_Gene_exon_count_1"]].drop_duplicates()
var_feature2 = data[["Variant_Gene_id_2", "Variant_Gene_name_2", "Variant_Gene_details_2", "Variant_Gene_chromosome_name_2", "Variant_Gene_chromosome_location_2", "Variant_Gene_exon_count_2"]].drop_duplicates()
var_feature1_renamed = var_feature1.rename(columns=lambda x: x.replace("_1", ""))
var_feature2_renamed = var_feature2.rename(columns=lambda x: x.replace("_2", ""))
combined_features = pd.concat(
    [var_feature1_renamed, var_feature2_renamed], 
    axis=0,  # 纵向合并
    ignore_index=True  # 重置索引
)
var_features = combined_features.drop_duplicates()

gene_id = var_features['Variant_Gene_id']
gene_name = var_features['Variant_Gene_name']
variation_details = var_features['Variant_Gene_details']
chromosome = var_features['Variant_Gene_chromosome_name']
chromosome_location = var_features['Variant_Gene_chromosome_location']
chromosome_exon_count = var_features['Variant_Gene_exon_count']

Variation_nodes = pd.DataFrame({
    "VariationId": variation_details,
    "gene_id": gene_id,
    "gene_name": gene_name,
    "variation_details": variation_details,
    "chromosome": chromosome,
    "chromosome_location": chromosome_location,
    "chromosome_exon_count": chromosome_exon_count,
    #":LABEL": "Variation"
})

In [3]:
#:START_ID,:END_ID,:TYPE
presents_relation = pd.DataFrame({
    "START_ID": pd.Series(dtype='str'),
    "END_ID": pd.Series(dtype='str'),
    "TYPE": pd.Series(dtype='str'),
})
diagnosed_with_relation = pd.DataFrame({
    "START_ID": pd.Series(dtype='str'),
    "END_ID": pd.Series(dtype='str'),
    "TYPE": pd.Series(dtype='str'),
})

fp_columns = data.filter(regex='^HP').columns
for i in range(len(data)):
    patient_id = data.loc[i]['patient_id']

    # presents
    row = data.loc[i, fp_columns]
    fp_ids_mask = (row != 0)
    fp_ids = row[fp_ids_mask].index.tolist()
    for fpid in fp_ids:
        new_row = pd.DataFrame({"START_ID": [patient_id], "END_ID": [fpid], "TYPE": ["presents"] })
        presents_relation = pd.concat([presents_relation, new_row], ignore_index=True)

    # diagnosed_with
    disease_id = data.loc[i]['Disease_id']
    new_row = pd.DataFrame({"START_ID": [patient_id], "END_ID": [disease_id], "TYPE": ["diagnosed_with"] })
    diagnosed_with_relation = pd.concat([diagnosed_with_relation, new_row], ignore_index=True)

diagnosed_with_relation = diagnosed_with_relation.dropna(axis=0)

C:\Users\15085\AppData\Local\Temp\ipykernel_22280\2691553354.py:28: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  diagnosed_with_relation = pd.concat([diagnosed_with_relation, new_row], ignore_index=True)


In [4]:
caused_by_relation = pd.DataFrame({
    "START_ID": pd.Series(dtype='str'),
    "END_ID": pd.Series(dtype='str'),
    "TYPE": pd.Series(dtype='str'),
})
leads_to_relation = pd.DataFrame({
    "START_ID": pd.Series(dtype='str'),
    "END_ID": pd.Series(dtype='str'),
    "TYPE": pd.Series(dtype='str'),
})

row_length = 0
fp_columns = data.filter(regex='^HP').columns
for var_id in var_features['Variant_Gene_details']:
    # var_id = var_features.loc[i]['Variant_Gene_details']

    # caused_by
    data_fliter = data[~data['Disease_id'].isnull()]
    disease_id = data_fliter.loc[(data_fliter['Variant_Gene_details_1'] == var_id) | (data_fliter['Variant_Gene_details_2'] == var_id),'Disease_id']
    for id in disease_id:
        new_row = pd.DataFrame({"START_ID": [var_id], "END_ID": [id], "TYPE": ["caused_by"] })
        caused_by_relation = pd.concat([caused_by_relation, new_row], ignore_index=True)
    
    # 这里有一部分重复统计了
    # leads_to
    rows = data.loc[(data['Variant_Gene_details_1'] == var_id) | (data['Variant_Gene_details_2'] == var_id), fp_columns]
    row_length += len(rows)
    for _, row in rows.iterrows():
        #row = rows.loc[i]
        fp_ids_mask = (row != 0)
        fp_ids = row[fp_ids_mask].index.tolist()
        for fpid in fp_ids:
            new_row = pd.DataFrame({"START_ID": [var_id], "END_ID": [fpid], "TYPE": ["leads_to"] })
            leads_to_relation = pd.concat([leads_to_relation, new_row], ignore_index=True)
leads_to_relation.drop_duplicates()

C:\Users\15085\AppData\Local\Temp\ipykernel_22280\21126050.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  caused_by_relation = pd.concat([caused_by_relation, new_row], ignore_index=True)


,START_ID,END_ID,TYPE
0,c.3838C > T; p. Arg1280Cys in exon 17,HP:0000252,leads_to
1,c.3838C > T; p. Arg1280Cys in exon 17,HP:0000340,leads_to
2,"c.456delC ,p.Gly153Alafs*34,in exon 1",HP:0000316,leads_to
3,"c.456delC ,p.Gly153Alafs*34,in exon 1",HP:0005280,leads_to
4,"c.456delC ,p.Gly153Alafs*34,in exon 1",HP:0000347,leads_to
...,...,...,...
8428,"c.154 G > A,p (Asp52Asn)",HP:0045075,leads_to
8429,"c.154 G > A,p (Asp52Asn)",HP:0000212,leads_to
8430,"c.154 G > A,p (Asp52Asn)",HP:0006316,leads_to
8431,"c.154 G > A,p (Asp52Asn)",HP:0045074,leads_to


In [5]:
hpo_nodes.to_csv("phenotype_nodes.csv", index=False)
disease_nodes.to_csv("disease_nodes.csv", index=False)
patient_nodes.to_csv("patient_nodes.csv", index=False)
Variation_nodes.to_csv("variation_nodes.csv", index=False)

In [6]:
presents_relation.to_csv("presents_relation.csv", index=False)
diagnosed_with_relation.to_csv("diagnosed_with_relation.csv", index=False)
caused_by_relation.to_csv("caused_by_relation.csv", index=False)
leads_to_relation.to_csv("leads_to_relation.csv", index=False)

# Putting csv files into the neo4j import directory

## run below cypher statments in neo4j

LOAD CSV WITH HEADERS FROM 'file:///disease_nodes.csv' AS row
CREATE (:Disease {
  DiseaseId: row.DiseaseId,
  name: row.name
});


LOAD CSV WITH HEADERS FROM 'file:///patient_nodes.csv' AS row
CREATE (:Patient {
  PatientId: row.PatientId,
  pmid: row.pmid,
  patient_number: row.patient_number,
  gender: row.gender,
  age: toFloat(row.age),
  region: row.region,
  race: row.race
});


LOAD CSV WITH HEADERS FROM 'file:///phenotype_nodes.csv' AS row
CREATE (:Phenotype {
  HpoId: row.HpoId,
  Facialphenotypename: row.Facialphenotypename
});


LOAD CSV WITH HEADERS FROM 'file:///variation_nodes.csv' AS row
CREATE (:Variation {
  VariationId: row.VariationId,
  gene_id: row.gene_id,
  gene_name: row.gene_name,
  variation_details: row.variation_details,
  chromosome: row.chromosome,
  chromosome_location: row.chromosome_location,
  chromosome_exon_count: row.chromosome_exon_count
});

LOAD CSV WITH HEADERS FROM 'file:///leads_to_relation.csv' AS row
MATCH (n1:Variation {VariationId: row.START_ID})
MATCH (n2:Phenotype {HpoId: row.END_ID})
CREATE (n1)-[r: leads_to]->(n2)
RETURN count(r);

LOAD CSV WITH HEADERS FROM 'file:///diagnosed_with_relation.csv' AS row
MATCH (n1:Patient {PatientId: row.START_ID})
MATCH (n2:Disease {DiseaseId: row.END_ID})
CREATE (n1)-[r: diagnosed_with]->(n2)
RETURN count(r);

LOAD CSV WITH HEADERS FROM 'file:///presents_relation.csv' AS row
MATCH (n1:Patient   {PatientId: row.START_ID})
MATCH (n2:Phenotype {HpoId: row.END_ID})
CREATE (n1)-[r: presents]->(n2)
RETURN count(r);

LOAD CSV WITH HEADERS FROM 'file:///caused_by_relation.csv' AS row
MATCH (n1:Disease {DiseaseId: row.END_ID})
MATCH (n2:Variation {VariationId: row.START_ID})
CREATE (n1)-[r: caused_by]->(n2)
RETURN count(r);